In [1]:
!pip install -U langchain-community tavily-python
!pip install -U langchain-openai langchainhub
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 21.1 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.71
    Uninstalling langsmith-0.1.71:
      Successfully uninstalled langsmith-0.1.71
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.4
    Uninstalling langchain-core-0.2.4:
      Successfully uninstalled langchain-core-0.2.4
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.2
    Uninstalling langchain-0.2.2:
      Successfully uninstalled langchain-0.2.2


  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.2
    Uninstalling langchain-community-0.2.2:
      Successfully uninstalled langchain-community-0.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.1.0 requires chromadb<0.5.0,>=0.4.0, but you have chromadb 0.5.0 which is incompatible.
langchain-chroma 0.1.0 requires langchain-core<0.2.0,>=0.1.40, but you have langchain-core 0.2.9 which is incompatible.
langchain-openai 0.1.3 requires langchain-core<0.2.0,>=0.1.42, but you have langchain-core 0.2.9 which is incompatible.
langchain-openai 0.1.3 requires openai<2.0.0,>=1.10.0, but you have openai 1.1.1 which is incompatible.


In [2]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

········


In [11]:
os.environ["OPENAI_API_KEY"] = "<ENTER YOUR OPENAI KEY>"



In [20]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [21]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [22]:
tool = TavilySearchResults(max_results=2)

In [23]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [34]:
prompt = """You are a stock analysts expert. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)



In [39]:
messages = [HumanMessage(content="What is the highest average dividend stock in Malaysia KLCI for the past 3 years?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='What is the best dividend stock in Malaysia KLCI ?', id='599c3927-eab1-4721-a618-57d7d01eb2e5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_igx3DdNw0QlVWDK1rQFgABpV', 'function': {'arguments': '{"query":"best dividend stock in Malaysia KLCI"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 158, 'total_tokens': 183}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d1984275-2527-4a08-a3b2-87e05fc6c7a7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'best dividend stock in Malaysia KLCI'}, 'id': 'call_igx3DdNw0QlVWDK1rQFgABpV'}]), ToolMessage(content="[{'url': 'http://www.dividendsranking.com/Index/FBMKLCI.php', 'content': 'FTSE Bursa Malaysia KLCI best dividend stocks and full list ranked by diviend yield ( Malaysia 4.34% average) It can be se

In [40]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the best dividend stock in Malaysia KLCI ?', id='599c3927-eab1-4721-a618-57d7d01eb2e5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_igx3DdNw0QlVWDK1rQFgABpV', 'function': {'arguments': '{"query":"best dividend stock in Malaysia KLCI"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 158, 'total_tokens': 183}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d1984275-2527-4a08-a3b2-87e05fc6c7a7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'best dividend stock in Malaysia KLCI'}, 'id': 'call_igx3DdNw0QlVWDK1rQFgABpV'}]), ToolMessage(content="[{'url': 'http://www.dividendsranking.com/Index/FBMKLCI.php', 'content': 'FTSE Bursa Malaysia KLCI best dividend stocks and full list ranked by diviend yield ( Malaysia 4.34% 

In [41]:
abot.graph.get_state(thread).next

('action',)

In [42]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'highest average dividend stock in Malaysia KLCI for the past 3 years'}, 'id': 'call_2VS5gXhuDTAASzxctQF3yIUv'}
Back to the model!
{'messages': [ToolMessage(content="[{'url': 'http://www.dividendsranking.com/Index/FBMKLCI.php', 'content': '08-31-2023. FTSE Bursa Malaysia KLCI best dividend stocks and full list ranked by diviend yield ( Malaysia 4.34% average) It can be seen, (Banks) - (Banks) - (Travel and Leisure) - are the companies that currently pay a higher dividend in the FTSE Bursa Malaysia Kuala Lumpur Composite Index, offering returns of 7.13% - 6.37% - 5.84% - respectively.'}, {'url': 'https://www.tradingview.com/markets/stocks-malaysia/market-movers-high-dividend/', 'content': 'Malaysian companies with the highest dividend yields. High dividends could signify a source of stable gains. Malaysian companies below pay the highest dividends in the market: see them sorted by forward dividend yield and check out othe